# The modular biochemical reaction network structure of cellular translation

## Abstract

Translation is an essential attribute of all living cells. At the heart of cellular operation, it is a chemical information decoding process that begins with an input string of nucleotides and ends with the synthesis of a specific output string of peptides. The translation process is interconnected with gene expression, physiological regulation, transcription, and responses to signaling molecules, among other cellular functions. Foundational efforts have uncovered a wealth of knowledge about the mechanistic functions of and many interactions between components of translation, but the broader biochemical connections between translation, metabolism and polymer biosynthesis that enable translation to occur have not been comprehensively mapped. Here we present a multilayer graph of biochemical reactions describing the translation, polymer biosynthesis and metabolism networks of an Escherichia coli cell. Intriguingly, the compounds that compose these three layers are distinctly aggregated into three modes regardless of their layer categorization. Multimodal mass distributions are well-known in ecosystems, but this is the first such distribution reported at the biochemical level. The connectivity distributions of the translation and metabolic networks are each likely to be heavy-tailed, but the polymer biosynthesis network is not. A multimodal mass-connectivity distribution indicates that the translation and metabolism networks are each distinct, adaptive biochemical modules, and that the gaps between the modes reflect evolved responses to the functional use of metabolite, polypeptide and polynucleotide compounds. The chemical reaction network of cellular translation opens new avenues for exploring complex adaptive phenomena such as percolation and phase changes in biochemical contexts. 



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import powerlaw as plw
import seaborn as sns
from Bio.SeqUtils import molecular_weight
from rdkit import Chem as chem
from rdkit.Chem.Descriptors import ExactMolWt
import json
import re

sns.set_style('darkgrid')

palette = dict(metabolite='#102ECC', protein='#596399', assembly='#64f195', RNA='#FFA86E', ion='#CC4010')
compound_palette = dict(metabolite='#102ECC', protein='#596399', assembly='#64f195', RNA='#FFA86E', ion='#CC4010')
palette_domain = dict(translation='#FF9E8C', metabolism='#298ACC', biosynthesis='#32A898')

## Flux between networks (Fig 3. B)

The following block retrieves how many compounds link each of the layers of the full network. We represented the numerical values of this analysis in Figure 3B.

In [ ]:
TMNI = nx.read_gml('networks/full-network.gml')

flux_dictionary = dict(
    metabolism = {"metabolism":0, "biosynthesis":0, "translation":0},
    biosynthesis = {"metabolism":0, "biosynthesis":0, "translation":0},
    translation = {"metabolism":0, "biosynthesis":0, "translation":0},
)


for node, node_data in TMNI.nodes(data=True):
    if node_data['type'] == 'chemical_entity':
        production_levels = []
        consumption_levels = []
        for reaction in TMNI.predecessors(node):
            i = TMNI.nodes()[reaction]['domain']
            if i not in production_levels: production_levels.append(i)
        for reaction in TMNI.successors(node):
            i = TMNI.nodes()[reaction]['domain']
            if i not in consumption_levels: consumption_levels.append(i)
        added = 0

        if len(production_levels) > 0 and len(consumption_levels) > 0:
            for p in production_levels:
                for c in consumption_levels:
                    flux_dictionary[p][c] += 1
        elif len(consumption_levels) > 0:
            for c in consumption_levels:
                flux_dictionary[c][c] += 1
        elif len(production_levels) > 0:
            for p in production_levels:
                flux_dictionary[p][p] += 1
        else:
            print(node)

np.set_printoptions(formatter={'float_kind':'{:6.3f}'.format})
pd.DataFrame.from_records(flux_dictionary)
        

## Statistical Network Properties: degree and mass distributions (Fig 5 A, B, C and D)

The following blocks provide the analysis of the two main attributes under study: the mass and the connectivity. For the analysis of the degree distribution, we extract the degree as the number of times that each compound interacts with reactions from a specific layer (or any of them, in the case of the full-network). For the mass, we rely in the mass that we have annotated for each of the compounds during the construction of the network.

In [ ]:
def extract_connectivity_values(G: nx.DiGraph, reaction_domain=None):
    out = []
    for node, node_data in G.nodes(data=True):
        if node_data['type'] == 'chemical_entity':
            
            degree_in = 0
            degree_out = 0
            for reaction in G.predecessors(node):
                domain = G.nodes()[reaction]['domain']
                if domain == reaction_domain or reaction_domain is None:
                    degree_in += 1
            for reaction in G.successors(node):
                domain = G.nodes()[reaction]['domain']
                if domain == reaction_domain or reaction_domain is None:
                    degree_out += 1

            if degree_in == 0 and degree_out == 0:
                continue
            else:
                
                try:
                    compound_type = node_data['compound_type']
                except KeyError:
                    compound_type = None

                try:
                    mass = float(node_data['mass'])
                except KeyError:
                    mass = None

                out.append(dict(
                    label=node,
                    degree_in = degree_in,
                    degree_out = degree_out,
                    degree_total = degree_in + degree_out,
                    weight = mass,
                    compound_type=compound_type,
                    domain=reaction_domain
                ))

    return pd.DataFrame.from_records(out)

In [ ]:
TMNI = nx.read_gml('networks/full-network.no-connections.gml')
translation_network_attributes = extract_connectivity_values(TMNI, reaction_domain='translation')
metabolism_network_attributes = extract_connectivity_values(TMNI, reaction_domain='metabolism')
biosynthesis_network_attributes = extract_connectivity_values(TMNI, reaction_domain='biosynthesis')
full_network_attributes = extract_connectivity_values(TMNI, reaction_domain=None)
full_network_attributes['domain'] = 'all'
network_attributes = pd.concat([translation_network_attributes, metabolism_network_attributes, biosynthesis_network_attributes, full_network_attributes])

In [ ]:
pd.DataFrame.from_records([dict(key=key, **data) for key, data in TMNI.nodes(data=True) if data['type'] == 'chemical_reaction'])

**Important note**: For some reason, the Powerlaw library fails to generate the right x-min values for the biosynthesis and the full network, generating very bad fits. We've manually tried different values to find a better fit to the data.

In [ ]:
discrete_fit = network_attributes.groupby('domain', as_index=False).apply(lambda x: plw.Fit(x.degree_total.values, discrete=True))
discrete_fit.columns = ['domain', 'discrete_fit']
continuous_fit = network_attributes.groupby('domain', as_index=False).apply(lambda x: plw.Fit(x.degree_total.values, discrete=False))

continuous_fit.columns = ['domain', 'continuous_fit']
fits = pd.merge(discrete_fit, continuous_fit, on='domain').set_index('domain')

fits.loc['all', 'continuous_fit'] = plw.Fit(network_attributes.query('domain == "all"').degree_total.values, discrete=False, xmin=3)
fits.loc['biosynthesis', 'continuous_fit'] = plw.Fit(network_attributes.query('domain == "biosynthesis"').degree_total.values, discrete=False, xmin=3)

fits['continuous_alpha'] = fits['continuous_fit'].apply(lambda x: x.alpha)
fits['discrete_alpha'] = fits['discrete_fit'].apply(lambda x: x.alpha)
fits['continuous_xmin'] = fits['continuous_fit'].apply(lambda x: x.xmin)
fits['discrete_xmin'] = fits['discrete_fit'].apply(lambda x: x.xmin)

fits['discrete_p_val_exp_powerlaw'] = fits['discrete_fit'].apply(lambda x: x.loglikelihood_ratio('exponential', 'power_law')[1])
fits['continuous_p_val_exp_powerlaw'] = fits['continuous_fit'].apply(lambda x: x.loglikelihood_ratio('exponential', 'power_law')[1])
fits['discrete_R_val_exp_powerlaw'] = fits['discrete_fit'].apply(lambda x: x.loglikelihood_ratio('exponential', 'power_law')[0])
fits['continuous_R_val_exp_powerlaw'] = fits['continuous_fit'].apply(lambda x: x.loglikelihood_ratio('exponential', 'power_law')[0])

# fits = fits.set_index('domain')
fits[['continuous_alpha', 'discrete_alpha', 'continuous_xmin', 'discrete_xmin', 'discrete_p_val_exp_powerlaw', 'continuous_p_val_exp_powerlaw']]


**Table 1**: Statistical fits of each of the degree distributions. The alpha value corresponds to the exponent of the fitted power-law.

In [ ]:
fig, ax = plt.subplots(3, 1, sharey=True, sharex=True)
fig.set_size_inches(3, 4)


fits.loc['translation'].continuous_fit.plot_ccdf(color='#FF9E8C', label='empirical', ax=ax[0], marker='.', linewidth=1)
fits.loc['translation'].continuous_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[0])
fits.loc['translation'].continuous_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[0])
fits.loc['biosynthesis'].continuous_fit.plot_ccdf(color='#32A898', label='empirical', ax=ax[1], marker='.', linewidth=1)
# fits.loc['biosynthesis'].continuous_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[1])
# fits.loc['biosynthesis'].continuous_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[1])
fits.loc['metabolism'].continuous_fit.plot_ccdf(color='#298ACC', label='empirical', ax=ax[2], marker='.', linewidth=1)
fits.loc['metabolism'].continuous_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[2])
fits.loc['metabolism'].continuous_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[2])

ax[0].set_ylim(10**-4, None)
ax[0].set_xticks([1, 10, 100, 1000])
ax[1].set_ylabel('CCDF(k)')
ax[2].set_xlabel('degree (k)')
ax[2].set_xlabel('degree (k)')
fig.tight_layout()
# plt.savefig('figures/power_law.all.svg', dpi=300)
# plt.savefig('figures/power_law.all_continuous.svg', dpi=300)


In [ ]:
fig, ax = plt.subplots(3, 1, sharey=True, sharex=True)
fig.set_size_inches(3, 4)


fits.loc['translation'].discrete_fit.plot_ccdf(color='#FF9E8C', label='empirical', ax=ax[0], marker='.', linewidth=1)
fits.loc['translation'].discrete_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[0])
fits.loc['translation'].discrete_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[0])
fits.loc['biosynthesis'].discrete_fit.plot_ccdf(color='#32A898', label='empirical', ax=ax[1], marker='.', linewidth=1)
fits.loc['biosynthesis'].discrete_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[1])
fits.loc['biosynthesis'].discrete_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[1])
fits.loc['metabolism'].discrete_fit.plot_ccdf(color='#298ACC', label='empirical', ax=ax[2], marker='.', linewidth=1)
fits.loc['metabolism'].discrete_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax[2])
fits.loc['metabolism'].discrete_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax[2])

ax[0].set_ylim(10**-4, None)
ax[0].set_xticks([1, 10, 100, 1000])
ax[1].set_ylabel('CCDF(k)')
ax[2].set_xlabel('degree (k)')
ax[2].set_xlabel('degree (k)')
fig.tight_layout()
# plt.savefig('figures/power_law.all.svg', dpi=300)
# plt.savefig('figures/power_law.all_discrete.svg', dpi=300)


In [ ]:
fig, ax = plt.subplots(1)
fig.set_size_inches(4, 4)
fits.loc['all'].continuous_fit.plot_ccdf(color='#6B8799', label='empirical', ax=ax, marker='o', linewidth=3)
fits.loc['all'].continuous_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax)
fits.loc['all'].continuous_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax)
ax.set_ylim(10**-4, None)
ax.set_xlabel('degree - k')
ax.set_ylabel('ccdf (k)')
# plt.savefig('figures/power_law.full_continuous.svg', dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1)
fig.set_size_inches(4, 4)
fits.loc['all'].discrete_fit.plot_ccdf(color='#6B8799', label='empirical', ax=ax, marker='o', linewidth=3)
fits.loc['all'].discrete_fit.power_law.plot_ccdf(color='black', label='power-law', ax=ax)
fits.loc['all'].discrete_fit.exponential.plot_ccdf(color='black', linestyle='--', label='exponential', ax=ax)
ax.set_ylim(10**-4, None)
ax.set_xlabel('degree - k')
ax.set_ylabel('ccdf (k)')
# plt.savefig('figures/power_law.full_discrete.svg', dpi=300)
plt.tight_layout()
plt.show()

**Figure 5.A** Degree distribution per layers. Y-axis represents the cumulative complementary distribution function, and the X-axis the of each compounds degree. The different plots divide the network by its layers.

**Figure S1**. Degree distribution of the full-network, using the same layout as in **Figure 5.A**

### Mass-degree distribution


**Important**. Although we can read the data from the network, there are some labels that require manual modification. Therefore, we just read it from a file.

In [ ]:
full_network_attributes = pd.read_csv('data/full-networks-components.csv', sep=';', index_col='index').sort_index()
full_network_attributes['log_weight'] = np.log10(full_network_attributes['weight'])
full_network_attributes['log_degree'] = np.log10(full_network_attributes['degree_total'])
network_attributes_by_domain = pd.concat([translation_network_attributes, metabolism_network_attributes, biosynthesis_network_attributes]).reset_index()
network_attributes_by_domain['logweight'] = np.log10(network_attributes_by_domain['weight'])

In [ ]:
# sns.set_palette('Set1')

fig, ax = plt.subplots(2, 2, sharex=True, sharey=False)
fig.set_size_inches(8, 5)
bins = np.arange(0, 8, 0.25)
# sns.kdeplot(data=full_network_attributes.query('compound_type != "placeholder"'), 
# hue='compound_type', x='log_weight', y='log_degree', palette=palette, alpha=0.85, ax=ax, levels=3, bw_adjust=1.25)
g = sns.scatterplot(
    data=full_network_attributes.query('compound_type != "placeholder"'), 
    hue='compound_type', x='log_weight', y='degree_total', palette=palette, ax=ax[0, 0],marker='.',
    linewidth=0
    
)
g = sns.histplot(
    x='log_weight', data=full_network_attributes.query('compound_type != "placeholder"'), 
    hue='compound_type', ax=ax[1, 0], palette=palette, bins=bins
)

g = sns.scatterplot(
    data=network_attributes_by_domain.query('compound_type != "placeholder"'), 
    hue='domain', x='logweight', y='degree_total', ax=ax[0,1], palette=palette_domain, marker='.',
    linewidth=0
)
g = sns.histplot(
    x='logweight', data=network_attributes_by_domain.query('compound_type != "placeholder"'), 
    hue='domain', ax=ax[1, 1], palette=palette_domain, bins=bins
)
ax[0, 0].set_yscale('log')
ax[0, 1].set_yscale('log')
ax[0, 0].set_ylabel('Degree')
ax[0, 1].set_ylabel('')
ax[1, 1].set_ylabel('')
ax[1, 1].set_xlabel('Weight (Da)')
ax[1, 0].set_xlabel('Weight (Da)')
ax[0, 1].set_xticklabels(['', '$10^{0}$', '$10^{2}$', '$10^{4}$','$10^{6}$'])
ax[1, 1].set_xticklabels(['', '$10^{0}$', '$10^{2}$', '$10^{4}$','$10^{6}$'])
ax[1, 0].set_ylim(1, 800)
ax[1, 1].set_ylim(1, 800)
# ax[0].set_yscale('log')
plt.tight_layout()
# plt.savefig('./figures/fig-mass-degree.svg')


## Network Embeddings 

In [ ]:
from torch_geometric.utils import from_networkx
from torch_geometric.nn import Node2Vec
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import torch

In [ ]:
TMN = nx.read_gml('networks/full-network.gml')
TMN_copy = TMN.copy()

In [ ]:
# Include biosynthesis

type_dict = dict(
    chemical_entity=[1, 0],
    chemical_reaction=[0, 1]
)
domain_dict = dict(
    metabolism = 1,
    translation = 2,
    biosynthesis = 3
)

def empty_node(x):
    x['x'] = type_dict[x['type']]
    try:
        x['y'] = domain_dict[x['domain']]
    except KeyError:
        x['y'] = 0

    keys = list(x.keys())
    for t in keys:
        if t not in ['x', 'y']:
            del x[t]


for node, node_data in TMN.nodes(data=True): empty_node(node_data)

for _, _, edge_data in TMN.edges(data=True):
    edge_data.clear()

data = from_networkx(TMN)
data.train_mask = torch.gt(torch.rand(len(data.x)), 0.75).to(torch.bool)
data.test_mask = (~ data.train_mask).to(torch.bool)
# data.y = torch.tensor([x['type'] for _, x in TMN.nodes(data=True)], dtype=torch.int32)

In [ ]:
# CHANGE TO CPU IF YOU DON'T HAVE A GPU
device = 'cuda:0'
model = Node2Vec(
    data.edge_index, embedding_dim=128, walk_length=20,
    context_size=10, walks_per_node=10,
    num_negative_samples=5, p=1, q=0.5, sparse=True
).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                        z[data.test_mask], data.y[data.test_mask],
                        max_iter=150)
    return acc

for epoch in range(1, 101):
    loss = train()
    acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc: {acc:.4f}')

In [ ]:
c = [node_data['y'] for _, node_data in TMN.nodes(data=True)]
z = model(torch.arange(data.num_nodes, device=device))
z = TSNE(n_components=2).fit_transform(z.detach().cpu().numpy())
# km = KMeans(n_clusters=16).fit(z)
# km.labels_

In [ ]:
c = np.array(c)
fig, ax = plt.subplots(1)
fig.set_size_inches(7, 7)
ax.set_aspect('equal')
ax.scatter(z[c==1, 0], z[c==1,1], c='#4DFFE6',  alpha=0.5, lw=0.25, edgecolors='black')
ax.scatter(z[c==2, 0], z[c==2,1], c='#FF9E8C',  alpha=0.5, lw=0.25, edgecolors='black')
ax.scatter(z[c==0, 0], z[c==0,1], c='gray',  alpha=0.5, lw=0.25, edgecolors='black')
ax.scatter(z[c==3, 0], z[c==3,1], c='#298ACC',  alpha=0.5, lw=0.25, edgecolors='black')
ax.set_xlabel('t-SNE (first component)')
ax.set_ylabel('t-SNE (second component)')
plt.savefig('figures/fig002.node2vec-tsne.svg', dpi=300)
plt.show()

In [ ]:
for i, (_, node_data) in enumerate(TMN_copy.nodes(data=True)):
    
    node_data['x'] = float(z[i, 0])
    node_data['y'] = float(z[i, 1])

nx.write_gml(TMN_copy, 'networks/full-network.node2vec.gml') 

## Network clustering

The following blocks prunes a metabolic-translation network to generate the network depicted in the Figure 4.

In [ ]:
MN = nx.read_gml('networks/metabolism.gml')
TN = nx.read_gml('networks/translation.gml')

In [ ]:
def group_bipartite(G, map):
    H = nx.DiGraph()
    for node, node_data in filter(lambda x: x[0] in map.keys(), G.nodes(data=True)):
        m = map[node]
        try:
            node_data = H.nodes[m]
        except KeyError:
            H.add_node(m, grouped=True, size=0, domain=node_data['domain']) #HARDCODED
            node_data = H.nodes[m]
        node_data = H.nodes()[m]
        node_data['size'] += 1

    for node, node_data in filter(lambda x: x[0] not in map.keys(), G.nodes(data=True)):
        H.add_node(node, grouped=False, **node_data)

    for start_node, end_node, _ in G.edges(data=True):

        try:
            _ = H.nodes[start_node]
        except KeyError:
            start_node = map[start_node]

        try:
            _ = H.nodes[end_node]
        except KeyError:
            end_node = map[end_node]

        try:
            edge_data = H.edges[start_node, end_node]
        except KeyError:
            H.add_edge(start_node, end_node, weight=0)
            edge_data = H.edges[start_node, end_node]
        edge_data['weight'] += 1
    H.remove_edges_from(nx.selfloop_edges(H))
    return H


def cluster_in(G, map):
    H = nx.DiGraph()
    map_values = list(map.values())
    for node, node_data in filter(lambda x: x[0] in map_values, G.nodes(data=True)):
        H.add_node(node, **node_data)

    for node, node_data in filter(lambda x: x[0] not in map_values, G.nodes(data=True)):
        interacts_with = list(set(list(G.pred[node]) + list(G.succ[node])))
        print(interacts_with)
        if len(interacts_with) == 1:
            if interacts_with in map_values:
                H[map[interacts_with]]['size'] += 1
        else:
            H.add_node(node, **node_data)


    for start_node, end_node, edge_data in G.edges(data=True):

        try:
            H.nodes[start_node]
            H.nodes[end_node]
        except KeyError:
            continue
        
        H.add_edge(start_node, end_node, **edge_data)
    
    return H
    

In [ ]:
module_map = dict()
domain_map = dict()
MTN = nx.compose(MN, TN)

for node, node_data in filter(lambda x: x[1]['type'] == "chemical_reaction", MTN.nodes(data=True)):
    try:
        module_map[node] = node_data['module']
    except KeyError:
        continue
    domain_map[node] =  node_data['domain']

In [ ]:
MTN_CG = group_bipartite(MTN, module_map)
MTN_CG = cluster_in(MTN_CG, module_map)

In [ ]:
nx.write_gml(MTN_CG, 'networks/coarse-grained-full-network.gml')